# DICOM TO JPEG WITH WINDOWING

In [ ]:
import os
import numpy as np
import pydicom
from pydicom.pixel_data_handlers import apply_windowing
from PIL import Image


def get_pixels_with_windowing(dcm_file):
    im = pydicom.dcmread(dcm_file)
    data = im.pixel_array
    data = apply_windowing(data, im)
    
    if im.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
    else:
        data = data - np.min(data)
        
    if np.max(data) != 0:
        data = data / np.max(data)
    data = (data * 255).astype(np.uint8)

    return data

def save_as_jpeg(output_file_path, data):
    # Convert the numpy array to a PIL Image
    image = Image.fromarray(data)
    
    # Save the image as JPEG
    image.save(output_file_path, format='JPEG')

def process_directory(input_dir, output_dir):
    for root, dirs, files in os.walk(input_dir):
        for file in files:
            if file.lower().endswith('.dcm'):
                input_file_path = os.path.join(root, file)
                relative_path = os.path.relpath(input_file_path, input_dir)
                output_file_path = os.path.join(output_dir, relative_path)
                
                # Ensure output directory exists
                output_file_dir = os.path.dirname(output_file_path)
                if not os.path.exists(output_file_dir):
                    os.makedirs(output_file_dir)

                # Process image
                pixels_with_windowing = get_pixels_with_windowing(input_file_path)
                
                # Save processed image as a JPEG file
                output_jpeg_path = output_file_path.replace('.dcm', '.jpeg')
                save_as_jpeg(output_jpeg_path, pixels_with_windowing) 

def main():
    input_base_dir = 'demo_dataset'
    output_base_dir = 'windowed_dicom'
    
    input_dir = os.path.join(input_base_dir)
    output_dir = os.path.join(output_base_dir)
    process_directory(input_dir, output_dir)

if __name__ == '__main__':
    main()


# MASK

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

def mask1(image):
    if image is None:
        return np.zeros_like(image), np.zeros_like(image)
    
    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(gray, (1, 1), 0)

    # Create a mask using adaptive thresholding
    mask = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2)

    # Apply morphological operations to remove small noises
    kernel = np.ones((120,120), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)

    # Find contours
    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

    if contours:
        # Select the largest contour
        largest_contour = max(contours, key=cv2.contourArea)

        # Create a new mask
        new_mask = np.zeros_like(gray)
        cv2.drawContours(new_mask, [largest_contour], -1, 255, thickness=cv2.FILLED)

        # Apply the mask
        result = cv2.bitwise_and(image, image, mask=new_mask)
    else:
        new_mask = np.zeros_like(gray)
        result = np.zeros_like(image)

    return new_mask, result

# Second mask creation function
def mask2(image):
    hh, ww = image.shape[:2]
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU)[1]
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))
    morph = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
    morph = cv2.morphologyEx(morph, cv2.MORPH_OPEN, kernel)
    contours = cv2.findContours(morph, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[0]
    if contours:
        big_contour = max(contours, key=cv2.contourArea)
        mask = np.zeros((hh, ww), dtype=np.uint8)
        cv2.drawContours(mask, [big_contour], 0, 255, cv2.FILLED)
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (25,25))
        mask = cv2.morphologyEx(mask, cv2.MORPH_DILATE, kernel)
        return mask
    else:
        return np.zeros((hh, ww), dtype=np.uint8)

# Main directories
input_dataset_path = 'windowed_dicom'
test_folder_name = 'val'
output_dataset_path = 'processed_dataset'

# Ensure the output directory exists
if not os.path.exists(output_dataset_path):
    os.makedirs(output_dataset_path)

# Traverse the input directory and process images
for root, dirs, files in os.walk(os.path.join(input_dataset_path)):
    for file_name in files:
        if file_name.lower().endswith('.jpeg'):
            # Read the image
            image_path = os.path.join(root, file_name)
            img = cv2.imread(image_path)
            
            # Create Mask1 and Mask2
            mask1_mask, mask1_result = mask1(img)
            mask2_mask = mask2(img)
            
            # Combine the two masks and get the result
            final_mask = cv2.bitwise_or(mask1_mask, mask2_mask)
            final_result = cv2.bitwise_and(img, img, mask=final_mask)

            # Create corresponding output path
            relative_path = os.path.relpath(root, input_dataset_path)
            output_folder = os.path.join(output_dataset_path, relative_path)
            
            if not os.path.exists(output_folder):
                os.makedirs(output_folder)

            # Save the final result
            output_file_path = os.path.join(output_folder, file_name)
            cv2.imwrite(output_file_path, final_result)




# CROP

In [ ]:
import os
import cv2
import torch
import shutil

class YOLOv5Model:
    def __init__(self, model_path):
        print("Initializing YOLOv5 Model...")
        self.model = torch.hub.load('./', 'custom', path=model_path, source='local')

    def detect_and_crop(self, image, image_name, save_path):
        try:
            # Perform detection using YOLOv5 model
            results = self.model(image)

            if results.xyxy[0].shape[0] == 0:
                # No objects detected, save the original image
                destination_path = os.path.join(save_path, image_name)
                cv2.imwrite(destination_path, image)
                return [destination_path]

            cropped_images = []

            for idx, det in enumerate(results.xyxy[0]):
                xmin, ymin, xmax, ymax = int(det[0]), int(det[1]), int(det[2]), int(det[3])

                # Crop the image based on bounding box
                cropped_image = image[ymin:ymax, xmin:xmax]

                # Save cropped image with original image name
                cropped_image_path = os.path.join(save_path, image_name)
                cv2.imwrite(cropped_image_path, cropped_image)

                cropped_images.append(cropped_image_path)

            return cropped_images

        except Exception as e:
            print(f"An error occurred during detection and cropping: {e}")
            # Save the original image if an error occurs during cropping
            destination_path = os.path.join(save_path, image_name)
            cv2.imwrite(destination_path, image)
            return [destination_path]

# Main script
if __name__ == "__main__":
    print("Script started...")

    # Initialize YOLOv5 model
    model_path = "Breast_Cropper.pt"  # Replace with your model path
    yolo_model = YOLOv5Model(model_path)

    # Path to folder containing images to process
    image_folder_path = "processed_dataset"

    # Path to folder where cropped images will be saved
    save_folder_path = "cropped_dataset"
    os.makedirs(save_folder_path, exist_ok=True)

    # Get list of images in the folder
    image_files = os.listdir(image_folder_path)
    image_files = [f for f in image_files if f.endswith('.jpeg') or f.endswith('.png')]  # Filter only JPEG/png images

    processed_count = 0

    for image_file in image_files:
        # Full path to the image
        image_path = os.path.join(image_folder_path, image_file)

        # Read the image
        image = cv2.imread(image_path)

        # Check if image is successfully loaded
        if image is not None:
            # Perform detection and cropping
            cropped_images = yolo_model.detect_and_crop(image, image_file, save_folder_path)
            processed_count += 1
        else:
            print(f"Error: Could not read image at {image_path}")

            # Copy the image to the destination folder as is
            destination_path = os.path.join(save_folder_path, image_file)
            os.makedirs(save_folder_path, exist_ok=True)
            shutil.copy(image_path, destination_path)
            processed_count += 1

    # Check if the number of processed images matches the total expected
    total_images = len(image_files)
    if processed_count == total_images:
        print(f"All {total_images} images processed successfully.")
    else:
        print(f"Error: Processed {processed_count} out of {total_images} images.")

    print("Script completed")


# CLAHE

In [ ]:
import os
import cv2

# Orijinal ve hedef ana klasör yolları
input_root = 'cropped_dataset'  # Orijinal klasör yolu
output_root = 'cropped_images_clahe'  # CLAHE uygulanmış görüntülerin kaydedileceği klasör

# CLAHE ayarları
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))

# Tüm alt klasörlerdeki görüntüleri dolaşma
input_folder = os.path.join(input_root)
output_folder = os.path.join(output_root)

# Klasör yapısını oluştur
os.makedirs(output_folder, exist_ok=True)

# Her bir klasördeki görüntüleri işle ve kaydet
for file in os.listdir(input_folder):
    if file.endswith(('.jpeg', '.jpg', '.png', '.bmp', '.tiff')):  # İşlenebilir görüntü dosya uzantıları
        # Orijinal görüntü dosya yolunu oluştur
        file_path = os.path.join(input_folder, file)
                
        # Görüntüyü yükle
        img = cv2.imread(file_path, 0)  # Gri tonlamalı olarak yükleme

        # CLAHE uygula
        clahe_img = clahe.apply(img)

        # Kaydedilecek hedef dosya yolunu oluştur
        output_path = os.path.join(output_folder, file)

        # CLAHE uygulanmış görüntüyü kaydet
        cv2.imwrite(output_path, clahe_img)

print("Tüm görüntüler CLAHE uygulanarak kaydedildi.")
